# Concept Properties

In [107]:
import pickle
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import nltk
from nltk.corpus import brown
lemmatizer = WordNetLemmatizer()

In [108]:
data = []
with open("../data/datasets/concept_properties/raw_data/norms.dat", "r") as f:
    for line in f.readlines()[1:]:
        domain, feature_type, concept, feature, freq =  line[:-1].split("\t")[:5]
        if feature[:2] == "is" and len(feature.split(" ")) == 2 and "_" not in concept:
            if "_" in feature.split(" ")[1]:
                features = feature.split(" ")[1].split("_")
                for f in features:
                    data.append((domain, feature_type, concept, f, int(freq)))
            else:
                data.append((domain, feature_type, concept.lower(), feature.split(" ")[1].lower(), int(freq)))

In [109]:
nouns = list(set([d[2] for d in data]))
props = list(set([d[3] for d in data]))

In [110]:
import pandas as pd

brysbaert_adjectives = []
brysbaert_data = pd.read_csv('/nlp/data/yueyang/prototypicality/clean/concreteness_predictor/Concreteness_ratings_Brysbaert_et_al_BRM.txt', delimiter='\t')
for i,row in brysbaert_data.iterrows():
    if row['Dom_Pos'] == 'Adjective':
        brysbaert_adjectives.append(row['Word'])

In [111]:
adjs = []
non_adjs = []
for prop in tqdm(props):
    try:
        top_pos = nltk.FreqDist(t for w, t in brown.tagged_words() if w.lower() == prop).most_common()[0][0]
    except:
        top_pos = 'JJ'
    if 'JJ' in top_pos:
        adjs.append(prop)
    else:
        non_adjs.append(prop)

100%|██████████| 802/802 [00:00<00:00, 2577.47it/s]


In [112]:
noun2props = {noun: [] for noun in nouns}
for d in tqdm(data):
    if d[2] in noun2props:
        if d[3] not in noun2props[d[2]] and d[3] not in non_adjs and d[-1] >= 3:
            noun2props[d[2]].append(d[3])
del_nouns = []
for noun, props in noun2props.items():
    if props == []:
        del_nouns.append(noun)
for noun in del_nouns:
    del noun2props[noun]

100%|██████████| 7334/7334 [00:00<00:00, 1351182.71it/s]


In [113]:
candidate_adjs = []
for noun, props in noun2props.items():
    candidate_adjs += props
candidate_adjs = list(set(candidate_adjs))

In [114]:
count = []
for noun, props in noun2props.items():
    count.append(len(props))

In [115]:
import numpy as np
np.mean(count)

8.590381426202322

In [10]:
# pickle.dump(noun2props, open("../data/datasets/concept_properties/noun2property/noun2prop.p", "wb"))

In [116]:
noun2det = pickle.load(open("inter_data/noun2det_CSLB.p", "rb"))

In [117]:
uncountable_nouns = []
with open("inter_data/uncountable_nouns.txt", "r") as f:
    for line in f.readlines():
        uncountable_nouns.append(line[:-1].lower()) 

In [118]:
prompt2templates = {"plural_most": ["most {} are [MASK].", "{} is [MASK]."], 
                    "singular_can_be": ["a {} can be [MASK].", "{} can be [MASK]."],
                    "singular_usually": ["a {} is usually [MASK].", "{} is usually [MASK]."], 
                    "singular_generally": ["a {} is generally [MASK].", "{} is generally [MASK]."],
                    "singular": ["a {} is [MASK].", "{} is [MASK]."],
                    "plural_all": ["all {} are [MASK].", "{} is [MASK]."],
                    "plural_can_be": ["{} can be [MASK].", "{} can be [MASK]."],
                    "plural_generally": ["{} are generally [MASK].", "{} is generally [MASK]."], 
                    "plural_some": ["some {} are [MASK].", "{} is [MASK]."], 
                    "plural_usually": ["{} are usually [MASK].", "{} is usually [MASK]."], 
                    "plural": ["{} are [MASK].", "{} is [MASK]."]}

In [129]:
import re,urllib,json
def countable(noun):
    thing = re.sub(' ', '\+', noun)
    url = 'https://books.google.com/ngrams/graph?content=many+' + thing + '%2C+much+' + thing + '&year_start=1980&year_end=2000'
    response = urllib.request.urlopen(url)
    html = response.read()
    thing = re.sub('\+', ' ', thing)
    for data in html.decode("utf-8").split("\n"):
        if "ngrams.data" in data and len(data) > 200:
            break
    try:
        many_data = json.loads(re.search('\{"ngram": "many ' + thing + '".*?\}', data, re.IGNORECASE).group(0))['timeseries']
        many = sum(many_data) / float(len(many_data))
    except:
        many = 0
    try:
        much_data = json.loads(re.search('\{"ngram": "much ' + thing + '".*?\}', data, re.IGNORECASE).group(0))['timeseries']
        much = sum(much_data) / float(len(much_data))
    except:
        much = 0
    return many > much

In [ ]:
import inflect
from tqdm import tqdm
p = inflect.engine()

In [105]:
for prompt_type, templates in prompt2templates.items():
    with open("../data/datasets/concept_properties/queries/" + prompt_type + ".prop", "w") as f:
        for noun in nouns:
            if lemmatizer.lemmatize(noun) not in uncountable_nouns:
                if "plural" in prompt_type:
                    plural_noun = p.plural(lemmatizer.lemmatize(noun))
                    sent = templates[0].format(plural_noun)
                else:
                    sent = templates[0].format(lemmatizer.lemmatize(noun))
                    det = noun2det[noun]
                    if det == "an":
                        sent = sent.replace("a ", "an ")
            else:
                sent = templates[1].format(lemmatizer.lemmatize(noun))
            f.write(noun + " :: " + sent + "\n")

In [121]:
import pickle
noun2props = pickle.load(open("../data/datasets/concept_properties/noun2property/noun2prop.p", "rb"))

In [122]:
count = 0
all_props = []
count_len = []
for noun, props in noun2props.items():
    count += len(props)
    all_props += props
    count_len.append(len(props))

In [123]:
from distutils.dir_util import copy_tree
import os
from tqdm import tqdm

In [124]:
import pickle
noun2prop_CSLB = pickle.load(open("../data/datasets/concept_properties/noun2property/noun2prop.p", "rb"))
noun2prop_MRD = pickle.load(open("../data/datasets/feature_norms/noun2property/noun2prop.p", "rb"))
noun2prop_memory_color = pickle.load(open("../data/datasets/memory_color/noun2property/noun2prop.p", "rb"))

In [125]:
CSLB_nouns = list(noun2prop_CSLB.keys())
MRD_nouns = list(noun2prop_MRD.keys())
memory_color_nouns = list(noun2prop_memory_color.keys())

In [126]:
valid_nouns = []
for noun in CSLB_nouns:
    if noun not in MRD_nouns and noun not in memory_color_nouns:
        valid_nouns.append(noun)

In [127]:
import random
test_nouns = random.sample(valid_nouns, 50)

In [128]:
noun2prop_train = {}
for noun in test_nouns:
    noun2prop_train[noun] = noun2prop_CSLB[noun]

In [11]:
pickle.dump(noun2prop_train, open("../data/datasets/concept_properties/noun2property/noun2prop_train.p", "wb"))

In [12]:
noun2prop_test = {}
for noun in CSLB_nouns:
    if noun not in test_nouns:
        noun2prop_test[noun] = noun2prop_CSLB[noun]

In [14]:
pickle.dump(noun2prop_test, open("../data/datasets/concept_properties/noun2property/noun2prop_test.p", "wb"))

# Feature Norms

In [ ]:
noun2prop = {}
with open("../data/datasets/feature_norms/raw_data/McRae_properties.gold", "r") as f:
    for raw_data in f.readlines():
        noun = raw_data.split("\t")[0]
        prop = raw_data.split("\t")[1][3:-1]
        if noun not in noun2prop:
            if '/' in prop:
                noun2prop[noun] = prop.split('/')
            else:
                noun2prop[noun] = [prop]
        else:
            if '/' in prop:
                noun2prop[noun] += prop.split('/')
            else:
                noun2prop[noun].append(prop)

# Memory Colors

In [ ]:
import inflect
import nltk
import copy
from nltk.stem.wordnet import WordNetLemmatizer
engine = inflect.engine()
Lem = WordNetLemmatizer()

In [ ]:
with open("../data/datasets/memory_color/raw_data/memory_color.txt", "r") as f:
    data = []
    for line in f.readlines():
        data.append(line[:-1].split("\t"))

In [ ]:
singulars = []
plurals = []
item2color = {}
for descriptor, item, color in data:
    s = Lem.lemmatize(item)
    p = engine.plural(s)
    singulars.append(s)
    plurals.append(p)
    item2color[item] = color

In [ ]:
with open("inter_data/singular_plural.txt", "r") as f:
    ps_data = []
    for line in f.readlines():
        ps_data.append(line[:-1].split("\t"))

In [ ]:
final_data = []
for i, (descriptor, item, color) in enumerate(data):
    final_data.append({"descriptor": descriptor, "item raw": item, "item singular": ps_data[i][0], "item plural": ps_data[i][1], "countable": ps_data[i][2], "color": color})

In [ ]:
prompt2templates = {"plural_most": ["most {} are [MASK].", "{} is [MASK]."], 
                    "singular_can_be": ["a {} can be [MASK].", "{} can be [MASK]."],
                    "singular_usually": ["a {} is usually [MASK].", "{} is usually [MASK]."], 
                    "singular_generally": ["a {} is generally [MASK].", "{} is generally [MASK]."],
                    "singular": ["a {} is [MASK].", "{} is [MASK]."],
                    "plural_all": ["all {} are [MASK].", "{} is [MASK]."],
                    "plural_can_be": ["{} can be [MASK].", "{} can be [MASK]."],
                    "plural_generally": ["{} are generally [MASK].", "{} is generally [MASK]."], 
                    "plural_some": ["some {} are [MASK].", "{} is [MASK]."], 
                    "plural_usually": ["{} are usually [MASK].", "{} is usually [MASK]."], 
                    "plural": ["{} are [MASK].", "{} is [MASK]."]}

In [ ]:
for prompt_type, templates in prompt2templates.items():
    with open("../data/datasets/memory_color/noun2property/queries/" + prompt_type + ".prop", "w") as f:
        for data in final_data:
            if data["countable"] == "1":
                if "plural" in prompt_type:
                    if len(data["descriptor"]) < 5:
                        sent = templates[0].format(data['item plural'])
                    else:
                        descriptor = copy.deepcopy(data["descriptor"])
                        if descriptor == "the animal":
                            sent = templates[0].format(descriptor + " " + data['item plural'])
                        else:
                            if descriptor == "the outside of a":
                                descriptor = "the outside of"
                            elif descriptor == "the inside of a":
                                descriptor = "the inside of"
                            sent = descriptor + " " + templates[0].format(data['item plural'])
                else:
                    if len(data["descriptor"]) <= 1:
                        sent = templates[0].format(data['item singular'])
                    elif data["descriptor"] == "an":
                        sent = templates[0].format(data['item singular']).replace("a ", "an ")
                    else:
                        descriptor = copy.deepcopy(data["descriptor"])
                        if descriptor == "the outside of a":
                            descriptor = "the outside of"
                        elif descriptor == "the inside of a":
                            descriptor = "the inside of"
                        sent = descriptor + " " + templates[0].format(data['item singular'])
            else:
                sent = templates[1].format(data['item raw'])
                if data['item raw'] == "jeans":
                    sent = sent.replace(" is ", " are ")
            f.write(data['item raw'] + " :: " + sent + "\n")

In [ ]:
noun2prop = {}
for data in final_data:
    noun2prop[data['item raw']] = [data['color']]

In [ ]:
import json
item2image_url = {}
with open("../data/datasets/memory_color/raw_data/memory_colors.jsonl", "rb") as f:
    for line in f.readlines():
        data = json.loads(line.decode('utf-8'))
        item2image_url[data['item']] = data['picture']

In [ ]:
import requests
from tqdm import tqdm

In [ ]:
for item, url in tqdm(item2image_url.items()):
    try:
        img_data = requests.get(url).content
        with open("../data/datasets/memory_color/images/gt_images/" + item + '.png', 'wb') as handler:
            handler.write(img_data)
    except:
        continue

In [ ]:
from simple_image_download import simple_image_download as simp
from PIL import Image
import requests
import io
from requests.exceptions import Timeout
response = simp.simple_image_download

In [ ]:
done_images = [f.split(".")[0] for f in os.listdir("../data/datasets/memory_color/images/gt_images/")]

In [ ]:
for item, url in tqdm(item2image_url.items()):
    if item in done_images:
        continue
    else:
        try:
            img_data = requests.get(url).content
            with open('png/{}.png'.format(item), 'wb') as handler:
                handler.write(img_data)
        except:
            continue

In [ ]:
img_data = requests.get(url).content
with open('image_name.png', 'wb') as handler:
    handler.write(img_data)